
# Name= R.Ramakrishna Raju
# Batch= 09012021
## Topic:  Naïve Bayes
## PYTHON 

# importing required libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

# importing Dataset

In [4]:
tweets=pd.read_csv('Disaster_tweets_NB.csv')
tweets


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
df1 = []
df2 = []
df1 = tweets[tweets['target'] > 0]
df2 = tweets[tweets['target'] <= 0]






In [7]:
df1 = df1[['text','target']]

In [8]:
df2 = df2[['text','target']]

In [9]:
train_xxx = df1[1:2617]#2617
train_yyy = df1[2618:3271]
train_xx = df2[1:3474]#3474
train_yy = df2[3475:4342]
train_x = pd.concat([train_xx, train_xxx])
test_y = pd.concat([train_yy, train_yyy])
data = pd.concat([train_x,test_y])

In [10]:
train_x.size

12178

In [11]:
test_y.size

3040

In [13]:
import re
stop_words = []
# Load the custom built Stopwords
with open(r"F:\360\NB\stopwords.txt","r") as sw:
    stop_words = sw.read()
stop_words = stop_words.split("\n")
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+"," ",i).lower()
    i = re.sub("[0-9" "]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word)>3:
            w.append(word)
    return (" ".join(w))

In [15]:
# testing above function with sample text => removes punctuations, numbers
cleaning_text("Hope you are having a good week. Just checking in")

'hope having good week just checking'

In [16]:

cleaning_text("hope i can understand your feelings 123121. 123 hi how .. are you?")


'hope understand your feelings'

In [17]:
cleaning_text("Hi how are you, I am good")


'good'

In [18]:
train_x.text = train_x.text.apply(cleaning_text)
test_y.text = test_y.text.apply(cleaning_text)

In [22]:
from sklearn.model_selection import train_test_split
def split_into_words(i):
    return [word for word in i.split(" ")]
# Defining the preparation of email texts into word count matrix format - Bag of Words
data_bow = CountVectorizer(analyzer = split_into_words).fit(data.text)
# Defining BOW for all messages
all_matrix = data_bow.transform(data.text)
# For training messages
train_matrix = data_bow.transform(train_x.text)
# For testing messages
test_matrix = data_bow.transform(test_y.text)
# Learning Term weighting and normalizing on entire emails
tfidf_transformer = TfidfTransformer().fit(all_matrix)
# Preparing TFIDF for train emails
train_tfidf = tfidf_transformer.transform(train_matrix)
train_tfidf.shape # (row, column)
# Preparing TFIDF for test emails
test_tfidf = tfidf_transformer.transform(test_matrix)
test_tfidf.shape #  (row, column)

(1520, 32010)

In [23]:
# Preparing a naive bayes model on training data set 
from sklearn.naive_bayes import MultinomialNB as MB

In [27]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_tfidf, train_x.target)
# Evaluation on Test Data
test_pred_m = classifier_mb.predict(test_tfidf)
accuracy_test_m = np.mean(test_pred_m == test_y.target)
accuracy_test_m
from sklearn.metrics import accuracy_score
accuracy_score(test_pred_m, test_y.target) 


0.7723684210526316

In [28]:
pd.crosstab(test_pred_m, test_y.target)

target,0,1
row_0,,
0,714,193
1,153,460


In [25]:
# Training Data accuracy
train_pred_m = classifier_mb.predict(train_tfidf)
accuracy_train_m = np.mean(train_pred_m == train_x.target)
accuracy_train_m


0.8516997865002464